In [1]:
# Styling notebook
from IPython.core.display import HTML
def css_styling():
    styles = open("./styles/custom.css", "r").read()
    return HTML(styles)
css_styling()

In [2]:
ops = ['STORE', 'PLUS', 'MINUS', 'TIMES', 'DIV', 'INCR', 'EQ?', 'NEQ?', 'BR', 'BZ', 'BNZ', 'DUP', 'POP', 'DISP']

# i = 1; while i != 10 { i++; }
code1 = ['i', 1, 'STORE', 'L1:', 'i', 10, 'NEQ?', 'L2!', 'BZ', 'i', 'INCR', 'L1!', 'BR', 'L2:', 'DISP']

# s = 0; i = 1; while i != 11 { s = s + i; i++; }
code2 = ['s', 0, 'STORE', 'i', 1, 'STORE', 'L1:', 'i', 101, 'NEQ?', 'L2!', 'BZ', 's' , 's', 'i', 'PLUS', 'STORE', 'i', 'INCR', 'L1!', 'BR', 'L2:', 'DISP']

In [3]:
class PostfixVM :
    def __init__(self, ops, code) :
        self.stack = None                    # Working stack
        self.env = None                      # Bindings
        self.ops = ops                       # List of op symbols
        self.code = self.typedCode(code)     # Code to execute: type hints added
        self.CONT = -1                       # Continuation counter
    
    def PUSH(self,value) : 
        self.stack.append(value)
    
    def POP(self) :
        return self.stack.pop()
    
    def ensureBound(self,key) :
        value = self.env.get(key,None)
        if (value != None) : return value
        self.env[key] = 0
        return 0
    
    def STORE(self,key,value) :
        self.env[key[1]] = value
        
    def EVAL(self, atom) :
        if atom[0] == 'NUM' : return atom[1] # constants evaluate to themselves
        # Must of type LOC now: Make sure it's in the environment and return the value
        return self.ensureBound(atom[1])
        
    def opType(self,op) :
        if isinstance(op, (float, int)) : return ['NUM',op]
        if op in self.ops : return ['OP',op]
        if op[-1] == ':'  : return ['LDEF', op[0:-1]]
        if op[-1] == '!'  : return ['LREF', op[0:-1]]
        return ['LOC', op]
    
    def typedCode(self,untypedCode) :
        return list(map(self.opType,untypedCode))  
    
    def display(self) :
        print(self.CONT)
        print(self.env)
        print(self.stack)
    
    def run(self) :
        self.CONT = 0;
        self.stack = []
        self.env = {}
        while self.CONT < len(self.code) : # Still inside the code
            op = self.code[self.CONT]
            self.CONT += 1
            self.doOp(op)
            # self.display()

    def position(self,lref) :
        return self.code.index(['LDEF',lref[1]])
            
    def doOp(self,op) :
        # print('OP: ',op)
        
        # Stack OPs
        
        if op[0] in ['LOC', 'NUM', 'LREF'] : self.PUSH(op)

        if op[0] == 'OP' :
            if op[1] == 'DISP' : self.display()
            if op[1] == 'STORE' :
                v = self.EVAL(self.POP())
                self.STORE(self.POP(),v)
                
            # Arithmetic
                
            if op[1] == 'PLUS' :
                x = self.EVAL(self.POP())
                self.PUSH(['NUM',self.EVAL(self.POP()) + x])
            if op[1] == 'TIMES' :
                x = self.EVAL(self.POP())
                self.PUSH(['NUM',self.EVAL(self.POP()) * x])
            if op[1] == 'MINUS' :
                x = self.EVAL(self.POP())
                self.PUSH(['NUM',self.EVAL(self.POP()) - x])
            if op[1] == 'DIV' :
                x = self.EVAL(self.POP())
                self.PUSH(['NUM',self.EVAL(self.POP()) / x])

            if op[1] == 'INCR' :
                var = self.POP()
                self.STORE(var,self.EVAL(var) + 1)
            if op[1] == 'DECR' :
                var = self.POP()
                self.STORE(var,self.EVAL(var) - 1)
                
            # Comparisons
                
            if op[1] == 'EQ?' :
                v = self.EVAL(self.POP()) == self.EVAL(self.POP())
                self.PUSH(['NUM',1 if v else 0])
            if op[1] == 'NEQ?' :
                v = self.EVAL(self.POP()) == self.EVAL(self.POP())
                self.PUSH(['NUM',0 if v else 1])
                
            # Branches
                
            if op[1] == 'BR' :
                self.CONT = self.position(self.POP())
            if op[1] == 'BZ' :
                label = self.POP()
                if self.EVAL(self.POP()) == 0 :
                    self.CONT = self.position(label)
            if op[1] == 'BNZ' :
                label = self.POP()
                if self.EVAL(self.POP()) != 0 :
                    self.CONT = self.position(label)
        return
        

In [5]:
pv = PostfixVM(ops,code2)
print(pv.code)
pv.run()

[['LOC', 's'], ['NUM', 0], ['OP', 'STORE'], ['LOC', 'i'], ['NUM', 1], ['OP', 'STORE'], ['LDEF', 'L1'], ['LOC', 'i'], ['NUM', 101], ['OP', 'NEQ?'], ['LREF', 'L2'], ['OP', 'BZ'], ['LOC', 's'], ['LOC', 's'], ['LOC', 'i'], ['OP', 'PLUS'], ['OP', 'STORE'], ['LOC', 'i'], ['OP', 'INCR'], ['LREF', 'L1'], ['OP', 'BR'], ['LDEF', 'L2'], ['OP', 'DISP']]
23
{'s': 5050, 'i': 101}
[]
